<a href="https://colab.research.google.com/github/guicorazza/Finance/blob/main/Investment_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Portfolio Risk
Let's download data for a portfolio of securities and calculate the risk of said portfolio.

#1. Installing libraries

In [ ]:
!pip install yfinance --upgrade --no-cache-dir
#https://pypi.org/project/yfinance/

In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt

Defining function to show results in percentage format

In [ ]:
def percentify(a):
  print(str(round(a,5) * 100) + '%')

We'll use Yahoo Finance API to retrieve the data. We'll override the yfinance data retrieve method to keep the data in a pandas dataframe format.

In [ ]:
import yfinance as yf
yf.pdr_override()

#2. Calculating security risk

Let's define the tickers to be analysed. Because I am Brazilian, I'll focus on brazilian stocks and indices. We'll use Vale and Petrobras, two of the most traded stocks in Brazil.

We'll focus on the Adjusted Close information only, as this column takes into account the dividends, splits and merges.

Obs.: tickers can be defined either in a tuple (with or without commas) or in a list.

In [ ]:
tickers = ['VALE3.SA', 'PETR4.SA']
sec_data = pd.DataFrame()
sec_data = wb.get_data_yahoo(tickers,period='5y')['Adj Close']
sec_data.tail()

[*********************100%***********************]  2 of 2 completed


,PETR4.SA,VALE3.SA
Date,,
2023-01-03,22.340000,89.239998
2023-01-04,23.049999,89.400002
2023-01-05,23.879999,90.900002
2023-01-06,23.740000,92.339996
2023-01-09,23.850000,92.500000


Calculating log returns of the securities:

In [ ]:
sec_returns = np.log(sec_data / sec_data.shift(1))
percentify(sec_returns)

            PETR4.SA  VALE3.SA
Date                          
2018-01-09       NaN       NaN
2018-01-10    -1.360    -1.403
2018-01-11     2.643     1.935
2018-01-12     0.289     0.576
2018-01-15     0.289    -0.184
...              ...       ...
2023-01-03    -2.563    -0.179
2023-01-04     3.129     0.179
2023-01-05     3.538     1.664
2023-01-06    -0.588     1.572
2023-01-09     0.462     0.173

[1240 rows x 2 columns]%


##2.1. Calculating annualized volatility (risk)

Let's get the annualized volatility for each security.

In [ ]:
print('Annual average return is: \n', sec_returns.mean()*252)

Annual average return is: 
 PETR4.SA    0.235179
VALE3.SA    0.228875
dtype: float64


In [ ]:
print('Standard deviation (volatility) is: \n', sec_returns.std())

Standard deviation (volatility) is: 
 PETR4.SA    0.031412
VALE3.SA    0.026020
dtype: float64


Since standard deviation is the square root of the variance, to annualize it you must multiply by: $\sqrt{252}$

$var \cdot 252 = s^2 \cdot 252$

$\sqrt{var \cdot 252} = \sqrt{s^2 \cdot 252}$

$= s \cdot \sqrt{252}$

where $s$ is the standard deviation.

We are using $s$ instead of $\sigma$, because we're using a sample of the data, not the whole population.

In [ ]:
print('Annualized volatility (risk) is: \n', sec_returns.std()*252 ** 0.5)

Annualized volatility (risk) is: 
 PETR4.SA    0.498643
VALE3.SA    0.413055
dtype: float64


#3. Covariance and Correlation

##3.1. Covariance
The formula for covariance is as below.

> $Cov_{xy}=\frac{(x-\bar{x})\cdot(y-\bar{y})}{n-1}$

Basically, the covariance is a qualitative measure of relationship between two variables, meaning:

Positive covariance -> they move in the same direction

Negative covariance -> they move in separate directions

Zero covariance -> they are independent

$Covariance\ Matrix = \sum
\begin{bmatrix}
\sigma_1^2 & \sigma_{12} & \cdots & \sigma_{1n}\\
\sigma_{21} & \sigma_2^2 & \cdots & \sigma_{2n}\\
\vdots & \vdots & \ddots & \vdots\\
\sigma_{n1} & \sigma_{n2} & \cdots & \sigma_n^2
\end{bmatrix}$

In this case, working with Ibovespa and Petrobras, we end up with the following matrix:
$\begin{array}{c|c}
  var(Vale) & covar(Vale, Petr) \\ 
  \hline
  covar(Vale, Petr) & var(Petr)
 \end{array}$
 
This means we need to calculate both the variance and covariance of the securities.

###3.1.1. Calculating variance

In [ ]:
Petr_var = sec_returns['PETR4.SA'].var()
Vale_var = sec_returns['VALE3.SA'].var()

print('Variance of Petrobras:',Petr_var)
print('Variance of Vale:',Vale_var)

Variance of Petrobras: 0.000986684652838594
Variance of Vale: 0.0006770399979753269


In [ ]:
Petr_var_a = Petr_var * 252
Vale_var_a = Vale_var * 252

print('Annualized variance of Petrobras:',Petr_var_a)
print('Annualized variance of Vale:',Vale_var_a)

Annualized variance of Petrobras: 0.24864453251532567
Annualized variance of Vale: 0.17061407948978238


###3.1.2. Calculating covariance
There is already a method that calculates the covariance matrix, so there is no need to calculate the variances previously. But it is good to check if the numbers match.

In [ ]:
cov_matrix = sec_returns.cov()
cov_matrix_a = cov_matrix*252
cov_matrix_a

,PETR4.SA,VALE3.SA
PETR4.SA,0.248645,0.090166
VALE3.SA,0.090166,0.170614


##3.2. Correlation coefficient
The formula for correlation is as below.

> $\rho_{xy} = \frac{(x-\bar{x})\cdot(y-\bar{y})}{\sigma_x\sigma_y}$

Differently from the qualitative aspect of the covariance, the correlation is quantitative, meaning it measures how strong is the linear relationship between two variables.

Its value ranges from -1 to 1, meaning:

Correlation = 1 → perfect positive correlation

Correlation = -1 → perfect negative correlation

Correlation = 0 → there's no correlation

In [ ]:
corr_matrix = sec_returns.corr()
corr_matrix

,PETR4.SA,VALE3.SA
PETR4.SA,1.000000,0.437772
VALE3.SA,0.437772,1.000000


The values above show the stocks are mildly correlated. It's expected some correlation, as there are macro factors that influence both, such as the BRL exchange rate.

Pay attention in the distinction of corr(prices) vs corr(returns).

Corr(returns) reflects the dependence between price variation at different times and focuses on the returns of your portfolio.

Corr(prices) focuses on stock price levels.

**Corr(returns) is what we, as investors, need**.

#4. Portfolio variance
Let's have an example of a portfolio with 2 stocks, we need the variance of that portfolio, so we can take the square root and find the standard deviation, i.e., the volatility of the portfolio.
<br><br>
###Portfolio with 2 stocks:

> $(w_1\sigma_1 + w_2\sigma_2)^2 = w_1^2\sigma_1^2 + 2w_1\sigma_1w_2\sigma_2\rho_{12}+w_2^2\sigma_2^2$

Where $w_1$ and $w_2$ are the weight of the stocks in the portfolio.

→ Also, $\sigma_1\sigma_2\rho_{12}$ = $Cov_{1,2}$, so we can replace that in the equation.

**This can be applied to any number of stocks, in which case we'll need to replicate this calculation for all combinations of pairs of stocks.**
<br><br>
###Portfolio with 3 stocks:

> $(w_1\sigma_1 + w_2\sigma_2 + w_3\sigma_3)^2 = w_1^2\sigma_1^2 + w_2^2\sigma_2^2 + w_3^2\sigma_3^2$ 

> $+ 2w_1w_2Cov_{12}$

> $+ 2w_1w_3Cov_{13}$

> $+ 2w_2w_3Cov_{23}$

##4.1. Calculating Portfolio Risk

###4.1.1. Portfolio variance: 

To calculate the variance, we can use an algebraic approach, which will make things easier, as it's easier to write and take advantage of numpy.

Arithmetic: $(ab)^2 = a^2b^2$

Algebraic: $(a\cdot B)^2 = a^TBa$

$(w \cdot Cov)^2 =
\begin{bmatrix}
w1 & w2
\end{bmatrix}
\begin{bmatrix}
v(1) & c(1,2)\\
c(1,2) & v(2)
\end{bmatrix}
\begin{bmatrix}
w1\\
w2
\end{bmatrix} = single\ number$

####Portfolio with 2 stocks
Let's calculate the portfolio risk. We'll begin with a 2 stock portfolio:

In [ ]:
weights = np.array([0.5,0.5])

In [ ]:
pfolio_var = np.dot(weights.T, np.dot(sec_returns.cov()*252, weights))
pfolio_var

0.14989781739060382

Portfolio volatility:

In [ ]:
pfolio_vol = pfolio_var**0.5
percentify(pfolio_vol)

38.717%


####Portfolio with 3 stocks

In [ ]:
weights_3 = np.array([1/3, 1/3, 1/3])

We've been using a portfolio with only 2 stocks so far, so let's include a third stock: MGLU3.

In [ ]:
tickers_3 = ['MGLU3.SA', 'PETR4.SA', 'VALE3.SA']

sec_data_3 = pd.DataFrame()
sec_data_3 = wb.get_data_yahoo(tickers_3,period='5y')['Adj Close']
sec_returns_3 = np.log(sec_data_3 / sec_data_3.shift(1))

pfolio_3_var = np.dot(weights_3.T, np.dot(sec_returns_3.cov()*252, weights_3))
pfolio_3_vol = pfolio_3_var**0.5

percentify(pfolio_3_vol)

[*********************100%***********************]  3 of 3 completed
38.353%


#5. Calculating Diversifiable and Non-Diversifiable Risk of a Portfolio

First, let's explain which are these risks:


*   Diversifiable risk (or idiosyncratic or unsystematic risk) is the one specific to the company or industry, which can be mitigated through diversification of non-correlated assets in the portfolio

*   Non-diversifiable risk (or systematic risk) is the one all companies are subject to, e.g., an economic recession, wars, natural disasters such as the recent pandemic, etc.

<img src=https://cdn.corporatefinanceinstitute.com/assets/Screen-Shot-2018-09-26-at-10.09.31-AM.png width=600>


##5.1. Diversifiable (unsystematic) Risk:
Because there are two types of risk, we can say that:
$Portfolio\ risk = Systematic\ risk + Diversifiable\ risk$

Moving one step back, we know that portfolio variance is:

$Portfolio\ var = w_1^2\sigma_1^2 + 2w_1\sigma_1w_2\sigma_2\rho_{12}+w_2^2\sigma_2^2$

Since the diversifiable risk is related to each company/industry, we can say that:

$2w_1\sigma_1w_2\sigma_2\rho_{12}$ = Diversifiable variance

$w_1^2\sigma_1^2 + w_2^2\sigma_2^2$ = Non-diversifiable variance

In [ ]:
diversifiable_var = pfolio_var - (weights[0]**2 * Petr_var_a) - (weights[1]**2 * Vale_var_a)
diversifiable_var

0.045083164389326795

##5.2. Non-Diversifiable Risk:
From the previous equation, we now know that:
$Systematic\ var = Portfolio\ var - Diversifiable\ var$

In [ ]:
#Calculation method 1:
non_diversifiable_var_1 = pfolio_var - diversifiable_var

#Calculation method 2:
non_diversifiable_var_2 = (weights[0]**2 * Petr_var_a) + (weights[1]**2 * Vale_var_a)

print(non_diversifiable_var_1)
print(non_diversifiable_var_2)

0.10481465300127701
0.10481465300127701
